In [ ]:
import matplotlib.pyplot as plt
from matplotlib.widgets import RadioButtons
import matplotlib.patches as patches
import re

def parse_input_file(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    grid_line = lines[0]
    grid_width, grid_height, *_ = map(int, grid_line.split(','))
    obs_lines, pin_lines = [], []
    for line in lines[1:]:
        if line.startswith("OBS"):
            obs_lines.append(line)
        else:
            pin_lines.append(line)
    obstacles = [tuple(map(int, re.search(r'\((\d+),\s*(\d+),\s*(\d+)\)', l).groups())) for l in obs_lines]

    pins, net_names = [], []
    for line in pin_lines:
        tokens = line.split()
        net_names.append(tokens[0])
        coords = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', line)
        pins.append([tuple(map(int, c)) for c in coords])
    return (grid_width, grid_height), obstacles, pins, net_names

def parse_output_file(filename):
    output_nets = []
    with open(filename, 'r') as f:
        for line in f:
            coords = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', line)
            output_nets.append([tuple(map(int, c)) for c in coords])
    return output_nets

def visualize_one_layer(grid_size, obstacles, input_nets, output_nets, net_names=None):
    grid_width, grid_height = grid_size
    all_layers = sorted(set(p[0] for net in output_nets for p in net))

    fig, ax = plt.subplots(figsize=(8, 8))
    plt.subplots_adjust(left=0.3)  # space for buttons

    def draw_layer(layer):
        ax.clear()
        ax.set_title(f"Layer {layer}")
        ax.set_xlim(-1, grid_width + 1)
        ax.set_ylim(-1, grid_height + 1)
        ax.set_aspect('equal')
        ax.set_xticks(range(0, grid_width, max(1, grid_width // 20)))
        ax.set_yticks(range(0, grid_height, max(1, grid_height // 20)))
        ax.grid(True, linestyle=':', linewidth=0.8)
        ax.set_facecolor('#f0f0f0')

        for (l, ox, oy) in obstacles:
            if l == layer:
                ax.add_patch(patches.Rectangle((ox, oy), 1, 1, color='black', zorder=2))

        for net_idx, net in enumerate(input_nets):
            for i, (l, x, y) in enumerate(net):
                if l == layer:
                    facecolor = '#90ee90' if i == 0 else 'green'
                    ax.add_patch(patches.Rectangle(
                        (x + 0.15, y + 0.15), 0.7, 0.7,
                        linewidth=1, edgecolor='black', facecolor=facecolor, zorder=4
                    ))

        cmap = plt.colormaps.get_cmap('tab20')
        for net_idx, net in enumerate(output_nets):
            color = cmap(net_idx)
            for (l, x, y) in net:
                if l == layer:
                    ax.add_patch(patches.Rectangle((x, y), 1, 1, color=color, zorder=3))

        ax.figure.canvas.draw_idle()

    draw_layer(all_layers[0])

    axcolor = 'lightgoldenrodyellow'
    rax = plt.axes([0.05, 0.4, 0.15, 0.2], facecolor=axcolor)
    radio = RadioButtons(rax, [str(l) for l in all_layers])

    def layer_changed(label):
        draw_layer(int(label))

    radio.on_clicked(layer_changed)
    plt.show()

if __name__ == "__main__":
    input_file = r"tests/input_500x500_1.txt"
    output_file = r"tests/output_500x500_1.txt"
    grid_size, obstacles, input_nets, net_names = parse_input_file(input_file)
    output_nets = parse_output_file(output_file)
    visualize_one_layer(grid_size, obstacles, input_nets, output_nets, net_names)

: 

In [1]:
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.patches as patches
import re
matplotlib.use('Qt5Agg')

def parse_input_file(file_path):
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f if line.strip()]
    grid_line = lines[0]
    grid_width, grid_height, *_ = map(int, grid_line.split(','))
    obs_lines, pin_lines = [], []
    for line in lines[1:]:
        if line.startswith("OBS"):
            obs_lines.append(line)
        else:
            pin_lines.append(line)
    obstacles = [tuple(map(int, re.search(r'\((\d+),\s*(\d+),\s*(\d+)\)', l).groups())) for l in obs_lines]

    pins, net_names = [], []
    for line in pin_lines:
        tokens = line.split()
        net_names.append(tokens[0])
        coords = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', line)
        pins.append([tuple(map(int, c)) for c in coords])
    return (grid_width, grid_height), obstacles, pins, net_names

def parse_output_file(filename):
    output_nets = []
    with open(filename, 'r') as f:
        for line in f:
            coords = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', line)
            output_nets.append([tuple(map(int, c)) for c in coords])
    return output_nets

def visualize(grid_size, obstacles, input_nets, output_nets, net_names=None):
    
    grid_width, grid_height = grid_size
    all_layers = sorted(set(p[0] for net in output_nets for p in net))
    cmap = plt.colormaps.get_cmap('tab20')
    
    fig, axs = plt.subplots(1, len(all_layers), figsize=(12 * len(all_layers), 12), dpi=150)
    if len(all_layers) == 1:
        axs = [axs]
    layer_to_ax = dict(zip(all_layers, axs))

    # Store artists per layer for toggling visibility
    layer_artists = {layer: [] for layer in all_layers}
    
    # Current visible layer (list so can modify in closure)
    current_layer = [all_layers[0]]

    # Initial axes and zoom/pan settings
    ax = layer_to_ax[current_layer[0]]

    def set_initial_limits():
        ax.set_xlim(-1, grid_width + 1)
        ax.set_ylim(-1, grid_height + 1)

    set_initial_limits()

    # Function to draw grid based on zoom level
    def draw_grid():
        ax.grid(False)
        xlim = ax.get_xlim()
        ylim = ax.get_ylim()
        x_range = xlim[1] - xlim[0]
        y_range = ylim[1] - ylim[0]
        # Determine grid step size dynamically
        step = max(1, int(min(x_range, y_range) // 20))
        ax.set_xticks(range(int(xlim[0]), int(xlim[1]) + 1, step))
        ax.set_yticks(range(int(ylim[0]), int(ylim[1]) + 1, step))
        ax.grid(True, linestyle=':', linewidth=0.8)

    # Draw static background and elements per layer
    for layer, ax_ in layer_to_ax.items():
        ax_.set_title(f"Layer {layer}")
        ax_.set_aspect('equal')
        ax_.set_facecolor('#f0f0f0')
        ax_.add_patch(patches.Rectangle(
            (0, 0), grid_width, grid_height,
            edgecolor='black', facecolor='white', linewidth=2, zorder=0
        ))
        draw_grid()

    # Prepare and store all artists for each layer
    for (layer, ox, oy) in obstacles:
        ax_ = layer_to_ax.get(layer)
        if ax_:
            r = patches.Rectangle((ox, oy), 1, 1, color='black', zorder=1)
            ax_.add_patch(r)
            layer_artists[layer].append(r)

    for net_idx, net in enumerate(output_nets):
        color = cmap(net_idx)
        net_label = net_names[net_idx] if net_names else f"net{net_idx+1}"
        layers_labeled = set()
        for (layer, x, y) in net:
            ax_ = layer_to_ax[layer]
            label = net_label if (net_label, layer) not in layers_labeled else None
            r = patches.Rectangle((x, y), 1, 1, color=color, zorder=1, label=label)
            ax_.add_patch(r)
            layer_artists[layer].append(r)
            layers_labeled.add((net_label, layer))

    used_via_layers = set()
    for net_idx, net in enumerate(output_nets):
        net_color = cmap(net_idx)
        for i in range(len(net) - 1):
            l1, x1, y1 = net[i]
            l2, x2, y2 = net[i + 1]
            if l1 != l2:
                for l in [l1, l2]:
                    ax_ = layer_to_ax.get(l)
                    if ax_:
                        label = "Via" if l not in used_via_layers else None
                        r = patches.Rectangle((x2 + 0.27, y2 + 0.27), 0.45, 0.45,
                                              linewidth=1.2, edgecolor='black',
                                              facecolor='red', zorder=6, label=label)
                        ax_.add_patch(r)
                        layer_artists[l].append(r)
                        if label is not None:
                            used_via_layers.add(l)

    used_light_pin_labels_per_layer = set()
    used_other_pin_labels_per_layer = set()

    for net_idx, net in enumerate(input_nets):
        for i, (layer, x, y) in enumerate(net):
            ax_ = layer_to_ax.get(layer)
            if ax_:
                if i == 0:
                    label_key = (layer, "Input Pin")
                    label = "Input Pin" if label_key not in used_light_pin_labels_per_layer else None
                    r = patches.Rectangle((x + 0.15, y + 0.15), 0.7, 0.7,
                                          linewidth=1, edgecolor='black',
                                          facecolor='#90ee90', zorder=4, label=label)
                    ax_.add_patch(r)
                    layer_artists[layer].append(r)
                    if label is not None:
                        used_light_pin_labels_per_layer.add(label_key)
                else:
                    label_key = (layer, "Output Pin")
                    label = "Output Pin" if label_key not in used_other_pin_labels_per_layer else None
                    r = patches.Rectangle((x + 0.15, y + 0.15), 0.7, 0.7,
                                          linewidth=1, edgecolor='black',
                                          facecolor='green', zorder=4, label=label)
                    ax_.add_patch(r)
                    layer_artists[layer].append(r)
                    if label is not None:
                        used_other_pin_labels_per_layer.add(label_key)

    # Hide all layers initially except current_layer
    for layer, ax_ in layer_to_ax.items():
        if layer != current_layer[0]:
            ax_.set_visible(False)

    def draw_layer():
        nonlocal ax
        for layer, ax_ in layer_to_ax.items():
            if layer == current_layer[0]:
                ax_.set_visible(True)
                # Redraw grid on current zoom
                draw_grid()
            else:
                ax_.set_visible(False)
        ax = layer_to_ax[current_layer[0]]
        set_initial_limits()
        fig.canvas.draw_idle()

    def set_initial_limits():
        ax.set_xlim(-1, grid_width + 1)
        ax.set_ylim(-1, grid_height + 1)

    # Zoom and pan event handlers
    def on_scroll(event):
        base_scale = 1.2
        ax = layer_to_ax[current_layer[0]]
        cur_xlim = ax.get_xlim()
        cur_ylim = ax.get_ylim()
        xdata = event.xdata
        ydata = event.ydata
        if xdata is None or ydata is None:
            return
        if event.button == 'up':
            scale_factor = 1 / base_scale
        elif event.button == 'down':
            scale_factor = base_scale
        else:
            scale_factor = 1
        new_width = (cur_xlim[1] - cur_xlim[0]) * scale_factor
        new_height = (cur_ylim[1] - cur_ylim[0]) * scale_factor
        relx = (xdata - cur_xlim[0]) / (cur_xlim[1] - cur_xlim[0])
        rely = (ydata - cur_ylim[0]) / (cur_ylim[1] - cur_ylim[0])
        ax.set_xlim([xdata - new_width * relx, xdata + new_width * (1 - relx)])
        ax.set_ylim([ydata - new_height * rely, ydata + new_height * (1 - rely)])
        draw_grid()
        fig.canvas.draw_idle()

    def on_press(event):
        ax = layer_to_ax[current_layer[0]]
        ax._pan_start = (event.x, event.y, ax.get_xlim(), ax.get_ylim())

    def on_motion(event):
        ax = layer_to_ax[current_layer[0]]
        if not hasattr(ax, '_pan_start') or ax._pan_start is None:
            return
        if event.x is None or event.y is None:
            return
        x_press, y_press, xlim, ylim = ax._pan_start
        dx = event.x - x_press
        dy = event.y - y_press
        # Calculate scale in data coords
        scale_x = (xlim[1] - xlim[0]) / ax.bbox.width
        scale_y = (ylim[1] - ylim[0]) / ax.bbox.height
        ax.set_xlim(xlim[0] - dx * scale_x, xlim[1] - dx * scale_x)
        ax.set_ylim(ylim[0] - dy * scale_y, ylim[1] - dy * scale_y)

        draw_grid()
        fig.canvas.draw_idle()

    def on_release(event):
        ax = layer_to_ax[current_layer[0]]
        ax._pan_start = None

    def on_key(event):
        nonlocal ax
        if event.key == 'left' or event.key == 'a':
            idx = all_layers.index(current_layer[0])
            current_layer[0] = all_layers[idx - 1] if idx > 0 else all_layers[-1]
            ax = layer_to_ax[current_layer[0]]
            set_initial_limits()
            draw_layer()
        elif event.key == 'right' or event.key == 'd':
            idx = all_layers.index(current_layer[0])
            current_layer[0] = all_layers[(idx + 1) % len(all_layers)]
            ax = layer_to_ax[current_layer[0]]
            set_initial_limits()
            draw_layer()

    # Connect event handlers
    fig.canvas.mpl_connect('scroll_event', on_scroll)
    fig.canvas.mpl_connect('button_press_event', on_press)
    fig.canvas.mpl_connect('motion_notify_event', on_motion)
    fig.canvas.mpl_connect('button_release_event', on_release)
    fig.canvas.mpl_connect('key_press_event', on_key)

    # Build legend from one axis (first layer visible)
    handles, labels = axs[0].get_legend_handles_labels()
    # Use a dict to keep only unique labels
    legend_handles = {}
    for h, l in zip(handles, labels):
        if l not in legend_handles:
            legend_handles[l] = h
    fig.legend(legend_handles.values(), legend_handles.keys(), loc='center right', fontsize='small', borderaxespad=0.1)

    plt.tight_layout(rect=[0, 0, 0.85, 1])
    plt.show()

            

if __name__ == "__main__":
    input_file = r"tests\input_500x500_1.txt"
    output_file = r"tests\output_500x500_1.txt"
    # input_file = r"tests_2\input_5x5_2.txt"
    # output_file = r"tests_2\output_5x5_2.txt"
    grid_size, obstacles, input_nets, net_names = parse_input_file(input_file)
    output_nets = parse_output_file(output_file)
    visualize(grid_size, obstacles, input_nets, output_nets, net_names)